In [ ]:
import pandas as pd #用於匯入、操作表格資料之函式庫
#用於資料可視化之函式庫
import matplotlib.pyplot as plt
import seaborn as sns 
import sklearn #機器學習演算法函式庫

train = pd.read_csv('../input/mobile-price-classification/train.csv')
test = pd.read_csv('../input/mobile-price-classification/test.csv')


<font size="5">查看資料欄位大致狀況</font>

In [ ]:
train.head() #使用head()查看前5筆資料，預設值為5

In [ ]:
test.drop('id',axis=1,inplace=True) #刪除id欄位，因為是多餘的
test.head()  #使用head()查看前5筆資料，預設值為5

<font size=5>查看目標欄位(特徵)在訓練集中的資料分布</font>
<p>從此專案得知我們需要根據其他特徵來預測手機價格範圍，先從訓練集中的price_range欄位查看資料數量的分布狀況</p>

In [ ]:
sns.countplot(train['price_range']) #查看目標特徵中資料在價錢範圍的分布

In [ ]:
train.shape,test.shape #訓練集資料總筆數為2000筆、測試集資料總筆數為1000筆，欄位數皆為21行

<font size=5>在進行模型訓練前，檢查資料是否含有空值，並且查看各欄位詳細狀況，接著了解各欄位之間的relation</font>

In [ ]:
"""
透過insnull()檢測訓練集中的資料是否有空值回傳一個
含有真偽值資料的dataframe，如有空值回傳true，否則回傳false;
使用sum()來讓各欄位為基準加總真偽值，更清楚的顯示
各欄位的所有筆資料是否有不同真偽值存在
"""
train.isnull().sum()

In [ ]:
train.info() #查看訓練集中各欄位資料的類型

In [ ]:
test.info() #查看測試集中各欄位資料的類型

In [ ]:
train.describe() #描述2000筆資料在各欄位的資料筆數、平均值、標準差、最小值、25百分位的值、中位數、75百分位的值、最大值

In [ ]:
plt.figure(figsize=(20,20)) #設定熱力圖顯示的大小
sns.heatmap(train.corr(),annot=True,cmap=plt.cm.Blues) #透過seaborn檢查欄位關聯性，發現ram對於價格範圍的影響很大
plt.show()

<font size=5>訓練模型前先對訓練資料進行分割成要訓練的資料及測試的資料</font>

In [ ]:
X = train.drop('price_range',axis=1) #將price_range欄位剃除，只留下剩下的欄位用於對目標欄位:price_range進行訓練
y = train['price_range'] #設定目標欄位

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.1,random_state=0) #隨機切分驗證資料來防止模型過擬合

<font size="5">透過StandardScaler對訓練集和測試集進行標準化</font>
<p>因為透過對資料的標準化，讓原本單位不一樣的資料能映射到另一個空間而不改變其原本的資料分布，可以提升模型收斂的速度使訓練時間縮短</p>

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() #建立StandardScaler物件以對資料標準化
X_train = sc.fit_transform(X_train) #先使用fit_transform()對訓練集做fit和transform的動作，fit用於找到一些指標，比如標準差、平均值、最大最小值等來進行後續的transform
X_test = sc.transform(X_test) #使用transform()來依據前面的訓練集所找到的標準差、平均值等等指標來來進行transform，用來保證訓練集和測試集是在同一個標準來進行訓練和測試

<font size="5">使用classification_report函式庫計算出Recall、Precision、F1-score、Support以評估模型</font>

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

<font size="5">建立SVM模型</font>
<p>通過不同分類器來比較模型準確度</p>

In [ ]:
from sklearn import svm

linearSvcModel=svm.LinearSVC(C=1, max_iter=10000) #建立線性SVC分類器
linearSvcModel.fit(X_train, Y_train) #使用訓練集訓練模型
predicted_train_linear=linearSvcModel.predict(X_train) #使用訓練集預測
predicted_test_linear=linearSvcModel.predict(X_test) #使用測試集預測
print('linearSvcModel')
print('train:',predicted_train_linear,'\ntest:',predicted_test_linear)
accuracy1 = linearSvcModel.score(X_train, Y_train) #計算訓練集準確率
print('accuracy_train',accuracy1)
accuracy1 = linearSvcModel.score(X_test, Y_test) #計算測試集準確率
print('accuracy_test',accuracy1)
print('分類報告:\n',classification_report(Y_test,predicted_test_linear))
#使用參數average='macro'，期望輸出不受各類別樣本數影響
print('precision_score:',sklearn.metrics.precision_score(Y_test,predicted_test_linear,average='macro'))
print('recall_score:',sklearn.metrics.recall_score(Y_test,predicted_test_linear,average='macro'))
print('f1_score:',sklearn.metrics.f1_score(Y_test,predicted_test_linear,average='macro'))
print('混淆矩陣:')
plt.figure(figsize=(5,5))
sns.heatmap(confusion_matrix(Y_test, predicted_test_linear),annot=True,cmap=plt.cm.Blues)
plt.show()


svcModel=svm.SVC(kernel='linear', C=1) #建立核為線性的SVC分類器
svcModel.fit(X_train, Y_train) #使用訓練集訓練模型
predicted_train_linear_ker=svcModel.predict(X_train) #使用訓練集預測
predicted_test_linear_ker=svcModel.predict(X_test) #使用測試集預測
print('\nSvcModelWithLinear')
print('train:',predicted_train_linear_ker,'\ntest:',predicted_test_linear_ker)
accuracy2 = svcModel.score(X_train, Y_train) #計算訓練集準確率
print('accuracy_train',accuracy2)
accuracy2 = svcModel.score(X_test, Y_test) #計算測試集準確率
print('accuracy_test',accuracy2)
print('分類報告:\n',classification_report(Y_test,predicted_test_linear_ker))
#使用參數average='macro'，期望輸出不受各類別樣本數影響
print('precision_score:',sklearn.metrics.precision_score(Y_test,predicted_test_linear_ker,average='macro'))
print('recall_score:',sklearn.metrics.recall_score(Y_test,predicted_test_linear_ker,average='macro'))
print('f1_score:',sklearn.metrics.f1_score(Y_test,predicted_test_linear_ker,average='macro'))
print('混淆矩陣:')
plt.figure(figsize=(5,5))
sns.heatmap(confusion_matrix(Y_test, predicted_test_linear_ker),annot=True,cmap=plt.cm.Blues)
plt.show()

polyModel=svm.SVC(kernel='poly', degree=3, gamma='auto', C=1) #建立核為多項式轉換的SVC分類器
polyModel.fit(X_train, Y_train) #使用訓練集訓練模型
predicted_train_poly=polyModel.predict(X_train) #使用訓練集預測
predicted_test_poly=polyModel.predict(X_test) #使用測試集預測
print('\nPolyModel')
print('train:',predicted_train_poly,'\ntest:',predicted_test_poly)
accuracy3 = polyModel.score(X_train, Y_train) #計算訓練集準確率
print('accuracy_train',accuracy3)
accuracy3 = polyModel.score(X_test, Y_test) #計算測試集準確率
print('accuracy_test',accuracy3)
print('分類報告:\n',classification_report(Y_test,predicted_test_poly))
#使用參數average='macro'，期望輸出不受各類別樣本數影響
print('precision_score:',sklearn.metrics.precision_score(Y_test,predicted_test_poly,average='macro'))
print('recall_score:',sklearn.metrics.recall_score(Y_test,predicted_test_poly,average='macro'))
print('f1_score:',sklearn.metrics.f1_score(Y_test,predicted_test_poly,average='macro'))
print('混淆矩陣:')
plt.figure(figsize=(5,5))
sns.heatmap(confusion_matrix(Y_test, predicted_test_poly),annot=True,cmap=plt.cm.Blues)
plt.show()

rbfModel=svm.SVC(kernel='rbf', gamma=0.2, C=1) #建立核為高斯轉換的SVC分類器
rbfModel.fit(X_train, Y_train) #使用訓練集訓練模型
predicted_train_rbf=rbfModel.predict(X_train) #使用訓練集預測
predicted_test_rbf=rbfModel.predict(X_test) #使用測試集預測
print('\nRBFModel')
print('train:',predicted_train_rbf,'\ntest:',predicted_test_rbf)
accuracy4 = rbfModel.score(X_train, Y_train) #計算訓練集準確率
print('accuracy_train',accuracy4)
accuracy4 = rbfModel.score(X_test, Y_test) #計算測試集準確率
print('accuracy_test',accuracy4)
print('分類報告:\n',classification_report(Y_test,predicted_test_rbf))
#使用參數average='macro'，期望輸出不受各類別樣本數影響
print('precision_score:',sklearn.metrics.precision_score(Y_test,predicted_test_rbf,average='macro'))
print('recall_score:',sklearn.metrics.recall_score(Y_test,predicted_test_rbf,average='macro'))
print('f1_score:',sklearn.metrics.f1_score(Y_test,predicted_test_rbf,average='macro'))
print('混淆矩陣:')
plt.figure(figsize=(5,5))
sns.heatmap(confusion_matrix(Y_test, predicted_test_rbf),annot=True,cmap=plt.cm.Blues)
plt.show()

<font size="5">建立KNN模型</font>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knnModel = KNeighborsClassifier(n_neighbors=3) #建立K-近鄰模型
knnModel.fit(X_train, Y_train) #使用訓練集訓練模型
predicted_train = knnModel.predict(X_train) #使用訓練集預測
predicted_test = knnModel.predict(X_test) #使用測試集預測
print('train:',predicted_train,'\ntest:',predicted_test)
accuracy = knnModel.score(X_train, Y_train) #計算訓練集準確率
print('accuracy_train',accuracy)
accuracy = knnModel.score(X_test, Y_test) #計算測試集準確率
print('accuracy_test',accuracy)
print('分類報告:\n',classification_report(Y_test,predicted_test))
#使用參數average='macro'，期望輸出不受各類別樣本數影響
print('precision_score:',sklearn.metrics.precision_score(Y_test,predicted_test,average='macro'))
print('recall_score:',sklearn.metrics.recall_score(Y_test,predicted_test,average='macro'))
print('f1_score:',sklearn.metrics.f1_score(Y_test,predicted_test,average='macro'))
print('混淆矩陣:')
plt.figure(figsize=(5,5))
sns.heatmap(confusion_matrix(Y_test, predicted_test),annot=True,cmap=plt.cm.Blues)
plt.show()

<font size="5">建立Decision Tree</font>

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decisionTreeModel = DecisionTreeClassifier() #建立決策樹
decisionTreeModel.fit(X_train, Y_train)
predicted_train = decisionTreeModel.predict(X_train) #使用訓練集預測
predicted_test = decisionTreeModel.predict(X_test) #使用測試集預測
print('train:',predicted_train,'\ntest:',predicted_test)
accuracy = decisionTreeModel.score(X_train, Y_train) #計算訓練集準確率
print('accuracy_train',accuracy)
accuracy = decisionTreeModel.score(X_test, Y_test) #計算測試集準確率
print('accuracy_test',accuracy) 
print('分類報告:\n',classification_report(Y_test,predicted_test))
#使用參數average='macro'，期望輸出不受各類別樣本數影響
print('precision_score:',sklearn.metrics.precision_score(Y_test,predicted_test,average='macro'))
print('recall_score:',sklearn.metrics.recall_score(Y_test,predicted_test,average='macro'))
print('f1_score:',sklearn.metrics.f1_score(Y_test,predicted_test,average='macro'))
print('混淆矩陣:')
plt.figure(figsize=(5,5))
sns.heatmap(confusion_matrix(Y_test, predicted_test),annot=True,cmap=plt.cm.Blues)
plt.show()

<font size="5">建立Random Forest</font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

randomForestModel = RandomForestClassifier(n_estimators=100, criterion = 'gini') #建立Random Forest Classifier模型
randomForestModel.fit(X_train, Y_train) #使用訓練集訓練模型
predicted_train = randomForestModel.predict(X_train) #使用訓練集預測
predicted_test = decisionTreeModel.predict(X_test) #使用測試集預測
print('train:',predicted_train,'\ntest:',predicted_test)
accuracy = randomForestModel.score(X_train, Y_train) #計算訓練集準確率
print('accuracy_train',accuracy)
accuracy = randomForestModel.score(X_test, Y_test) #計算測試集準確率
print('accuracy_test',accuracy)
print('分類報告:\n',classification_report(Y_test,predicted_test))
#使用參數average='macro'，期望輸出不受各類別樣本數影響
print('precision_score:',sklearn.metrics.precision_score(Y_test,predicted_test,average='macro'))
print('recall_score:',sklearn.metrics.recall_score(Y_test,predicted_test,average='macro'))
print('f1_score:',sklearn.metrics.f1_score(Y_test,predicted_test,average='macro'))
print('混淆矩陣:')
plt.figure(figsize=(5,5))
sns.heatmap(confusion_matrix(Y_test, predicted_test),annot=True,cmap=plt.cm.Blues)
plt.show()